<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-maf-gradio/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAF (Microsoft Agent Framework)

Cell 1: Package Installation This cell is responsible for installing all the necessary Python libraries that the project needs to run. It includes agent-framework for building AI agents, gradio for creating a web-based user interface, openai for interacting with OpenAI's models, and nest_asyncio to handle asynchronous operations smoothly within Jupyter notebooks. The --pre flag is used to ensure any pre-release versions of these packages are also considered for installation if required.

Cell 2: Environment Setup and API Key Initialization This cell sets up the environment before the main logic begins. It imports various modules, including those for asynchronous programming, operating system interactions, Gradio, and Google Colab's user data storage. A crucial step here is applying a patch using nest_asyncio to make sure asynchronous code runs correctly in the Jupyter environment. It also filters out a specific warning message to keep the output clean. Most importantly, it securely retrieves your OpenAI API key from Colab's secrets manager and sets it as an environment variable, which the OpenAI client will automatically use.

Cell 3: Agent Definition and Chat Logic This cell contains the core logic for the AI agent:

Tool Definition: It defines an asynchronous Python function called calculate_force. This function takes mass and acceleration as input and returns the calculated force in Newtons. This function acts as a 'tool' that the AI agent can intelligently choose to use.
Client Initialization: It creates an instance of the OpenAI chat client, specifying the gpt-4o-mini model to be used. This client automatically picks up the OpenAI API key that was set in the previous cell.
Agent Creation: It then creates the AI agent itself, named "PhysicsBot." It provides instructions to the agent, telling it to act as a helpful physics assistant and to use its tools for calculations. The calculate_force function is registered as one of these tools.
Gradio Chat Logic: Finally, an asynchronous function chat_with_agent is defined. This function is designed to be the bridge between the Gradio interface and the AI agent. When a user types a message in the Gradio chat, this function will pass the message to the PhysicsBot agent, wait for the agent's response, and then return that response to be displayed in the chat interface.
Cell 4: Launching the Gradio Interface This final cell brings everything together by launching the user interface:

It creates a Gradio ChatInterface, which is a simple, chat-like web application. It tells Gradio to use the chat_with_agent function to handle all user inputs and agent responses.
It also sets a title for the application ("MAF Physics Assistant (Colab Edition)") and provides a brief description, encouraging users to ask physics questions or use the force calculator tool.
The demo.launch() command starts the web application. The share=True argument generates a public, temporary link so you can access the application from any web browser, and debug=True provides additional diagnostic information in the Colab console.


In [ ]:
!pip install agent-framework gradio openai nest_asyncio --pre

In [ ]:
import nest_asyncio
import os
import gradio as gr
import asyncio
from agent_framework.openai import OpenAIChatClient
from google.colab import userdata
import warnings

# Patch the event loop for Jupyter compatibility
nest_asyncio.apply()

# Filter specific DeprecationWarning from jupyter_client
warnings.filterwarnings("ignore", category=DeprecationWarning, module='jupyter_client')

# Initialize API key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
# 1. Define a tool
async def calculate_force(mass: float, acceleration: float):
    """Calculates force given mass (kg) and acceleration (m/s^2)."""
    return f"The calculated force is {mass * acceleration} Newtons."

# 2. Initialize the Client and Agent
client = OpenAIChatClient(model_id="gpt-4o-mini") # Automatically reads from OPENAI_API_KEY env var

agent = client.create_agent(
    name="PhysicsBot",
    instructions="You are a helpful physics assistant. Use tools for calculations.",
    tools=[calculate_force]
)

# 3. Define the Gradio Chat logic
async def chat_with_agent(message, history):
    # agent.run() is an async call
    response = await agent.run(message)
    return response.text

In [ ]:
# 4. Launch Gradio - share=True is recommended for a stable external link
demo = gr.ChatInterface(
    fn=chat_with_agent,
    title="MAF Physics Assistant (Colab Edition)",
    description="Ask physics questions or use the Force Calculator tool!"
)

demo.launch(share=True, debug=True)